In [1]:
import pandas as pd
pd.set_option('display.max_colwidth',100)
import nltk


In [2]:
t_data=pd.read_csv('drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/English/english_hope_train.csv',names=["text","label","NaN"], sep='\t')
t_data.pop('NaN')
t_data.head(9)

,text,label
0,these tiktoks radiate gay chaotic energy and i love it,Non_hope_speech
1,@Champions Again He got killed for using false money,Non_hope_speech
2,It's not that all lives don't matter,Non_hope_speech
3,Is it really that difficult to understand? Black lives matter and all lives matter are not mutua...,Non_hope_speech
4,Whenever we say black isn't that racists? Why don't just say Americans.,Non_hope_speech
5,Ros The Boss u don’t know that she’s actually lgbtq tho so....,Non_hope_speech
6,That was funny at the end when Larry said 'What are we arguing about then'. haha but that said,Non_hope_speech
7,She saves lives with her music.,Non_hope_speech
8,There were a lot of Samoans in my Army unit,Non_hope_speech


In [3]:
t_data['text']=t_data['text'].str.lower()       #changing into lower case       (remove and check acc too)

In [4]:
t_data['text']=t_data['text'].str.strip()       #remove white spaces
t_data['text']=t_data['text'].str.replace(r'\d+','')   #remove numbers
t_data['text']=t_data['text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))  #removing emoji
t_data['text']=t_data['text'].str.replace('[^\w\s]','')      #removing punct

In [5]:
#removing url if any
import re
def remove_URL(txt):
    url= re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",txt)
t_data['text']=t_data['text'].apply(lambda x:remove_URL(x))

In [6]:
#removing stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
def remove_sw(txt):
    no_sw =" ".join([a for a in txt.split() if a not in stop])
    return no_sw
t_data['text'] = t_data['text'].apply(lambda x: remove_sw(x))
t_data.head(9)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,label
0,tiktoks radiate gay chaotic energy love,Non_hope_speech
1,champions got killed using false money,Non_hope_speech
2,lives dont matter,Non_hope_speech
3,really difficult understand black lives matter lives matter mutually exclusive true general stat...,Non_hope_speech
4,whenever say black isnt racists dont say americans,Non_hope_speech
5,ros boss u dont know shes actually lgbtq tho,Non_hope_speech
6,funny end larry said arguing haha said,Non_hope_speech
7,saves lives music,Non_hope_speech
8,lot samoans army unit,Non_hope_speech


In [7]:
#correct wrong words
#from textblob import TextBlob
#t_data['text'] = t_data['text'].apply(lambda txt: ''.join(TextBlob(txt).correct()))    #(Taking so much time)

In [8]:
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
labelEncode=preprocessing.LabelEncoder()
labelEncode.fit(t_data['label'])
print (labelEncode.classes_)
train_labelEncode=labelEncode.transform(t_data['label'])
label=to_categorical(np.asarray(train_labelEncode))
label

['Hope_speech' 'Non_hope_speech' 'not-English']


array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [9]:
#counting labels      
 
t_data['label'].value_counts()
#must use something to balance data

Non_hope_speech    20778
Hope_speech         1962
not-English           22
Name: label, dtype: int64

In [10]:
#words in each line
 
t_data['totalwords'] = t_data['text'].str.count(' ') + 1
t_data

,text,label,totalwords
0,tiktoks radiate gay chaotic energy love,Non_hope_speech,6
1,champions got killed using false money,Non_hope_speech,6
2,lives dont matter,Non_hope_speech,3
3,really difficult understand black lives matter lives matter mutually exclusive true general stat...,Non_hope_speech,43
4,whenever say black isnt racists dont say americans,Non_hope_speech,8
...,...,...,...
22757,load bollocks every life matters simple,Non_hope_speech,6
22758,say lives matter deku wouldnt say way bro,Non_hope_speech,8
22759,god says life matters,Non_hope_speech,4
22760,video shit bunch whiny ass bitches bitching nothing,Non_hope_speech,8


In [11]:
import keras
from keras.preprocessing.text import Tokenizer
tok1 = Tokenizer(char_level=False, filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n', lower = True)

In [12]:
tok2=Tokenizer(char_level=True,filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n',lower=True)

In [13]:
tok1.fit_on_texts(t_data['text'])
words=len(tok1.word_counts)
words

20373

In [14]:
tok2.fit_on_texts(t_data['text'])

In [15]:
Charlist=tok2.word_index
#Charlist

In [16]:
wordlist=tok1.word_index
#wordlist

In [17]:
encode=tok1.texts_to_sequences(t_data['text'])
print(t_data['text'][5555])
encode[5555]

j brown another thing get angry said ask question many life white officers died protecting people could say blacklivesmatter died cuz live around dallas kid back blue lives matter saying life better


[586,
 407,
 148,
 56,
 13,
 1075,
 22,
 291,
 223,
 61,
 26,
 8,
 611,
 395,
 2383,
 3,
 121,
 9,
 1511,
 395,
 634,
 80,
 175,
 4664,
 146,
 82,
 690,
 1,
 2,
 19,
 26,
 110]

In [18]:
##         Padding encoded sequence of words
from keras.preprocessing import sequence
max_length=10
padd = sequence.pad_sequences(encode, maxlen=max_length, padding='post')
padd

array([[6571, 9564,  863, ...,    0,    0,    0],
       [9565,   87,   85, ...,    0,    0,    0],
       [   1,    5,    2, ...,    0,    0,    0],
       ...,
       [  20,  132,   26, ...,    0,    0,    0],
       [  59,  367,  478, ...,   93,    0,    0],
       [2554, 4209, 1069, ...,    0,    0,    0]], dtype=int32)

In [19]:
# importing libraries for creating model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding,MaxPool1D
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D,LSTM,Bidirectional

#Embedding layer

In [20]:
Embedding_Layer=Embedding(input_dim = words + 1, output_dim = 300,input_length=max_length)

#CNN

In [21]:
model = Sequential()
model.add(Embedding_Layer)
model.add(Conv1D(filters =256,kernel_size=2, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Conv1D(filters =512,kernel_size=3, activation='relu'))
model.add(MaxPooling1D(3,padding='same'))
model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
#model.add(Dense(4, activation = 'relu'))
model.add(Dense(3, activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 300)           6112200   
_________________________________________________________________
conv1d (Conv1D)              (None, 9, 256)            153856    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 4, 256)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2, 512)            393728    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 512)            0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                1

#LSTM

In [22]:
model_1 = Sequential()
model_1.add(Embedding_Layer)
model_1.add(LSTM(256,return_sequences=False))

#model_1.add(Flatten())
model_1.add(Dense(32, activation = 'relu'))
#model.add(Dense(4, activation = 'relu'))
model_1.add(Dense(3, activation = 'sigmoid'))
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 300)           6112200   
_________________________________________________________________
lstm (LSTM)                  (None, 256)               570368    
_________________________________________________________________
dense_2 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 6,690,891
Trainable params: 6,690,891
Non-trainable params: 0
_________________________________________________________________


#Bidirectional

In [23]:
model_2 = Sequential()
model_2.add(Embedding_Layer)
model_2.add(Bidirectional(LSTM(256,return_sequences=False)))

#model_2.add(Flatten())
model_2.add(Dense(32, activation = 'relu'))
#model.add(Dense(4, activation = 'relu'))
model_2.add(Dense(3, activation = 'sigmoid'))
model_2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 300)           6112200   
_________________________________________________________________
bidirectional (Bidirectional (None, 512)               1140736   
_________________________________________________________________
dense_4 (Dense)              (None, 32)                16416     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 99        
Total params: 7,269,451
Trainable params: 7,269,451
Non-trainable params: 0
_________________________________________________________________


#compile

In [24]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

#Training models

In [25]:
model.fit(padd,label,epochs=20,verbose=1,batch_size=64)

Epoch 1/20
356/356 [==============================] - 67s 187ms/step - loss: 0.2182 - acc: 0.9094
Epoch 2/20
356/356 [==============================] - 67s 187ms/step - loss: 0.1099 - acc: 0.9353
Epoch 3/20
356/356 [==============================] - 70s 196ms/step - loss: 0.0616 - acc: 0.9650
Epoch 4/20
356/356 [==============================] - 72s 203ms/step - loss: 0.0353 - acc: 0.9817
Epoch 5/20
356/356 [==============================] - 73s 204ms/step - loss: 0.0215 - acc: 0.9877
Epoch 6/20
356/356 [==============================] - 75s 210ms/step - loss: 0.0183 - acc: 0.9900
Epoch 7/20
356/356 [==============================] - 70s 198ms/step - loss: 0.0153 - acc: 0.9897
Epoch 8/20
356/356 [==============================] - 75s 211ms/step - loss: 0.0119 - acc: 0.9921
Epoch 9/20
356/356 [==============================] - 70s 196ms/step - loss: 0.0103 - acc: 0.9930
Epoch 10/20
356/356 [==============================] - 72s 201ms/step - loss: 0.0115 - acc: 0.9920
Epoch 11/20
356/356

In [26]:
model_1.fit(padd,label,epochs=20,verbose=1,batch_size=64)

Epoch 1/20
356/356 [==============================] - 51s 138ms/step - loss: 0.1612 - acc: 0.9245
Epoch 2/20
356/356 [==============================] - 49s 137ms/step - loss: 0.0443 - acc: 0.9755
Epoch 3/20
356/356 [==============================] - 48s 135ms/step - loss: 0.0320 - acc: 0.9811
Epoch 4/20
356/356 [==============================] - 48s 135ms/step - loss: 0.0221 - acc: 0.9873
Epoch 5/20
356/356 [==============================] - 48s 135ms/step - loss: 0.0213 - acc: 0.9860
Epoch 6/20
356/356 [==============================] - 48s 134ms/step - loss: 0.0148 - acc: 0.9900
Epoch 7/20
356/356 [==============================] - 48s 135ms/step - loss: 0.0137 - acc: 0.9915
Epoch 8/20
356/356 [==============================] - 49s 137ms/step - loss: 0.0121 - acc: 0.9928
Epoch 9/20
356/356 [==============================] - 49s 137ms/step - loss: 0.0117 - acc: 0.9918
Epoch 10/20
356/356 [==============================] - 49s 137ms/step - loss: 0.0132 - acc: 0.9914
Epoch 11/20
356/356

In [27]:
model_2.fit(padd,label,epochs=20,verbose=1,batch_size=64)

Epoch 1/20
356/356 [==============================] - 79s 212ms/step - loss: 0.1145 - acc: 0.9470
Epoch 2/20
356/356 [==============================] - 75s 210ms/step - loss: 0.0213 - acc: 0.9873
Epoch 3/20
356/356 [==============================] - 73s 206ms/step - loss: 0.0137 - acc: 0.9910
Epoch 4/20
356/356 [==============================] - 74s 207ms/step - loss: 0.0103 - acc: 0.9931
Epoch 5/20
356/356 [==============================] - 74s 208ms/step - loss: 0.0098 - acc: 0.9946
Epoch 6/20
356/356 [==============================] - 74s 208ms/step - loss: 0.0115 - acc: 0.9915
Epoch 7/20
356/356 [==============================] - 74s 208ms/step - loss: 0.0075 - acc: 0.9941
Epoch 8/20
356/356 [==============================] - 76s 212ms/step - loss: 0.0093 - acc: 0.9935
Epoch 9/20
356/356 [==============================] - 77s 217ms/step - loss: 0.0089 - acc: 0.9936
Epoch 10/20
356/356 [==============================] - 77s 217ms/step - loss: 0.0060 - acc: 0.9952
Epoch 11/20
356/356

#Development data

In [28]:
d_data=pd.read_csv('drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/English/english_hope_dev.csv',names=['text','label','nan'],sep='\t')

In [29]:
d_data=d_data[["text","label"]]
d_data

,text,label
0,Thats it.... like....I dont like that statue,Non_hope_speech
1,@Generation X Counting money that she been given for promoting hysterical green agenda.,Non_hope_speech
2,@Paola Hernandez i never said to be intolerant and hateful........ -_-,Non_hope_speech
3,@Firstlast300 Wow An opinion is that I don't like a color,Non_hope_speech
4,WOW!!!!!!!That was so so inspiring and incredible!!! THis speech has just saved my day,Hope_speech
...,...,...
2838,Such fake sentiment. .,Non_hope_speech
2839,@A G black lives arent undervalued compared to white lives though,Non_hope_speech
2840,People who pulled it down can and will be arrested for causing damage to public property lol.,Non_hope_speech
2841,@Aaron Castellanos It will be a two hour movie showing people watching television,Non_hope_speech


In [30]:
#processing as training set

d_data['text']=d_data['text'].str.lower()       #changing into lower case       (remove and check acc too)
d_data['text']=d_data['text'].str.strip()       #remove white spaces
d_data['text']=d_data['text'].str.replace(r'\d+','')   #remove numbers
d_data['text']=d_data['text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))  #removing emoji
d_data['text']=d_data['text'].str.replace('[^\w\s]','')      #removing punct
d_data['text']=d_data['text'].apply(lambda x:remove_URL(x))
d_data['text'] = d_data['text'].apply(lambda x: remove_sw(x))


In [44]:
encoded1 =tok1.texts_to_sequences(d_data['text'])
print(d_data['text'][0])
encoded1[0]

thats likei dont like statue


[18, 5, 6, 72]

In [45]:
padded = sequence.pad_sequences(encoded1, maxlen=max_length, padding='post')
padded[0]

array([18,  5,  6, 72,  0,  0,  0,  0,  0,  0], dtype=int32)

In [46]:
val_labelEncode=labelEncode.transform(d_data['label'])
val_label=to_categorical(np.asarray(val_labelEncode))
val_label[0]

array([0., 1., 0.], dtype=float32)

#prediction

In [47]:
# devlopment prediction of CNN

dev_predictions = model.predict(padded)
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
dev_predictions1 = np.zeros_like(dev_predictions)
dev_predictions1[np.arange(len(dev_predictions)), dev_predictions.argmax(1)] = 1
#print(dev_predictions)
#print(dev_predictions1)

In [48]:
# devlopment prediction of Lstm

l_dev_predictions = model_1.predict(padded)
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
l_dev_predictions1 = np.zeros_like(l_dev_predictions)
l_dev_predictions1[np.arange(len(l_dev_predictions)), l_dev_predictions.argmax(1)] = 1
#print(l_dev_predictions)
#print(l_dev_predictions1)

In [49]:
# devlopment prediction of BiLSTM

b_dev_predictions = model_2.predict(padded)
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
b_dev_predictions1 = np.zeros_like(b_dev_predictions)
b_dev_predictions1[np.arange(len(b_dev_predictions)), b_dev_predictions.argmax(1)] = 1
#print(b_dev_predictions)
#print(b_dev_predictions1)

#Accuracy report

In [50]:
#Accuracy CNN
from sklearn.metrics import classification_report
print(classification_report(val_label,dev_predictions1))

              precision    recall  f1-score   support

           0       0.50      0.30      0.38       272
           1       0.93      0.97      0.95      2569
           2       0.00      0.00      0.00         2

   micro avg       0.90      0.90      0.90      2843
   macro avg       0.48      0.42      0.44      2843
weighted avg       0.89      0.90      0.89      2843
 samples avg       0.90      0.90      0.90      2843



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [51]:
#Accuracy Lstm
print(classification_report(val_label,l_dev_predictions1))

              precision    recall  f1-score   support

           0       0.39      0.35      0.37       272
           1       0.93      0.94      0.94      2569
           2       0.00      0.00      0.00         2

   micro avg       0.89      0.89      0.89      2843
   macro avg       0.44      0.43      0.44      2843
weighted avg       0.88      0.89      0.88      2843
 samples avg       0.89      0.89      0.89      2843



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
#Accuracy BiiLstm
print(classification_report(val_label,b_dev_predictions1))

              precision    recall  f1-score   support

           0       0.35      0.40      0.38       272
           1       0.94      0.92      0.93      2569
           2       0.00      0.00      0.00         2

   micro avg       0.87      0.87      0.87      2843
   macro avg       0.43      0.44      0.44      2843
weighted avg       0.88      0.87      0.87      2843
 samples avg       0.87      0.87      0.87      2843



#Testing Data

In [53]:
tx_data=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/English/english_hope_test.csv',names=['text'],sep=',')
tx_data

,text
0,What do you mean by the word sniped?
1,I love this video!! I’m bisexual and it’s just really nice to hear that someone has gone through...
2,ya the irony but then i don't want to come off as a bigot myself
3,A PERSON'S CHARACTER MATTERS. PERIOD!!
4,@Blaster of Gasters
...,...
2841,+Ashrenneemakeup I think it's all a deliberate attempt to be controversial. We humans can't get ...
2842,Sheriff David Clarke. This guy is amazing.
2843,Abandorn Hope Situation
2844,Sheriff Clarke you are a person of such strong character and wisdom.nI want to see more of you a...


In [54]:
#PREprocessing
#processing as training set

tx_data['text']=tx_data['text'].str.lower()       #changing into lower case       (remove and check acc too)
tx_data['text']=tx_data['text'].str.strip()       #remove white spaces
tx_data['text']=tx_data['text'].str.replace(r'\d+','')   #remove numbers
tx_data['text']=tx_data['text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))  #removing emoji
tx_data['text']=tx_data['text'].str.replace('[^\w\s]','')      #removing punct
tx_data['text']=tx_data['text'].apply(lambda x:remove_URL(x))
tx_data['text'] = tx_data['text'].apply(lambda x: remove_sw(x))

In [55]:
encoded2 =tok1.texts_to_sequences(tx_data['text'])
padded2 = sequence.pad_sequences(encoded2, maxlen=max_length, padding='post')

In [56]:
#prediction
#prediction of CNN

predictions = model.predict(padded2)
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
predictions1 = np.zeros_like(predictions)
predictions1[np.arange(len(predictions)), predictions.argmax(1)] = 1
#print(dev_predictions)
#print(dev_predictions1)

In [57]:
labelEncode.inverse_transform(predictions1)

ValueError: ignored